In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import time
import datetime
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
from sklearn import utils
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from sklearn import decomposition
import math
from math import sqrt
import seaborn as sns
from datetime import timedelta  

# Handle table-like data and matrices
from sklearn.model_selection import GridSearchCV, cross_validate

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import mean_squared_error
# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

# Modelling Helpers
from sklearn.preprocessing import Imputer , Normalizer , scale
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
#models
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

#importing libraries
from sklearn.datasets import load_boston
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
full = train.append( test , ignore_index = True ) 
full['purchase_date'] = pd.to_datetime(full['purchase_date'] )
full['release_date'] = pd.to_datetime(full['release_date'] )
full['purchase_date'] = full['purchase_date'].fillna(full['release_date'])+ timedelta(days=30)
full['total_positive_reviews'].fillna((full['total_positive_reviews'].mean()), inplace=True)
full['total_negative_reviews'].fillna((full['total_negative_reviews'].mean()), inplace=True)
print(full.isnull().any())
full.head(5)

categories                False
genres                    False
id                        False
is_free                   False
playtime_forever           True
price                     False
purchase_date             False
release_date              False
tags                      False
total_negative_reviews    False
total_positive_reviews    False
dtype: bool


,categories,genres,id,is_free,playtime_forever,price,purchase_date,release_date,tags,total_negative_reviews,total_positive_reviews
0,"Single-player,Steam Trading Cards,Steam Cloud","Adventure,Casual,Indie",0,False,0.000000,3700.0,2018-08-01,2013-12-10,"Indie,Adventure,Story Rich,Casual,Atmospheric,...",96.0,372.0
1,"Single-player,Partial Controller Support",RPG,1,True,0.016667,0.0,2016-12-26,2015-08-12,"Mod,Utilities,RPG,Game Development,Singleplaye...",0.0,23.0
2,"Single-player,Full controller support,Steam Tr...","Adventure,Casual,Indie",2,False,0.000000,5000.0,2018-08-01,2014-01-28,"Point & Click,Adventure,Story Rich,Comedy,Indi...",663.0,3018.0
3,"Single-player,Multi-player,Steam Achievements,...","Action,RPG",3,False,1.533333,9900.0,2016-12-28,2010-03-31,"Medieval,RPG,Open World,Strategy,Sandbox,Actio...",1746.0,63078.0
4,"Single-player,Co-op,Steam Achievements,Full co...","Action,Indie,Strategy",4,False,22.333333,4800.0,2018-04-03,2012-07-30,"Tower Defense,Co-op,Action,Strategy,Online Co-...",523.0,8841.0


In [3]:
# train = pd.read_csv("train.csv")
# test = pd.read_csv("test.csv")
# train.dropna(how='any',inplace=True)
# test.dropna(how='any',inplace=True)
# train = train.reset_index()
# test = test.reset_index()
# full = train.append( test , ignore_index = True ) 
# print("Train data shape:",train.shape,"Test data shape",test.shape)
# full.tail()

In [4]:
#date_test = pd.concat((train['playtime_forever'],train['purchase_date'],train['release_date']),axis=1)
date_test = pd.concat((full['purchase_date'],full['release_date']),axis=1)

In [5]:
date_test.head()

,purchase_date,release_date
0,2018-08-01,2013-12-10
1,2016-12-26,2015-08-12
2,2018-08-01,2014-01-28
3,2016-12-28,2010-03-31
4,2018-04-03,2012-07-30


In [6]:
date_test['release_month'] = 0
date_test['release_year'] = 0
date_test['purchase_month'] = 0
date_test['purchase_year'] = 0

In [7]:
date_test['purchase_month'] = date_test['purchase_date'].map(lambda x: x.month)
date_test['purchase_year'] = date_test['purchase_date'].map(lambda x: x.year)
date_test['release_month'] = date_test['purchase_date'].map(lambda x: x.month)
date_test['release_year'] = date_test['purchase_date'].map(lambda x: x.year)
#或者 = date_test['purchase_date'].dt.year这样
#date_test['purchase_date'].dt.year

In [8]:
#date_test.drop(['purchase_date','release_date'],axis=1)
date_release_year = date_test['release_year']
date_release_year = pd.get_dummies(date_release_year,prefix='release_year')
date_release_year = date_release_year.drop(['release_year_2016'],axis=1)

date_release_month = date_test['release_month']
date_release_month = pd.get_dummies(date_test['release_month'],prefix='release_month')
date_release_month = date_release_month.drop(['release_month_1'],axis=1)

date_purchase_month = date_test['purchase_month']
date_purchase_month = pd.get_dummies(date_test['purchase_month'],prefix='purchase_month')
date_purchase_month = date_purchase_month.drop(['purchase_month_2'],axis=1)

date_purchase_year = date_test['purchase_year']
date_purchase_year = pd.get_dummies(date_test['purchase_year'],prefix='purchase_year')
date_purchase_year = date_purchase_year.drop(['purchase_year_2015'],axis=1)

In [9]:
date_release_year.head()

,release_year_2011,release_year_2015,release_year_2017,release_year_2018,release_year_2019
0,0,0,0,1,0
1,0,0,0,0,0
2,0,0,0,1,0
3,0,0,0,0,0
4,0,0,0,1,0


In [10]:
date_test['purchase_timestamp'] = date_test["purchase_date"].values.astype(np.int64) // 10 ** 9
date_test['release_timestamp'] = date_test["release_date"].values.astype(np.int64) // 10 ** 9

In [11]:
date_test['purchase_timestamp'].describe()

count    4.470000e+02
mean     1.523206e+09
std      2.684976e+07
min      1.316045e+09
25%      1.511482e+09
50%      1.523491e+09
75%      1.539778e+09
max      1.570406e+09
Name: purchase_timestamp, dtype: float64

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
date_test[['purchase_timestamp','release_timestamp']] = scaler.fit_transform(date_test[['purchase_timestamp','release_timestamp']])
Timestamp = pd.DataFrame()
Timestamp['purchase_timestamp'] = date_test['purchase_timestamp']
Timestamp['release_timestamp'] = date_test['release_timestamp']
# Timestamp['purchase_timestamp'] =   1569603286 - date_test['purchase_timestamp']
Timestamp.head()

,purchase_timestamp,release_timestamp
0,0.853261,0.564352
1,0.655231,0.693480
2,0.853261,0.574725
3,0.655910,0.278577
4,0.812500,0.458933


In [13]:
# date_test['gap_purchase_release'] = date_test['purchase_timestamp'] - date_test['release_timestamp']
# date_test.head()

In [14]:
# to be used for date
date = pd.concat([date_purchase_year, date_purchase_month, date_release_year,date_release_month, Timestamp],axis=1)
date.head()

,purchase_year_2011,purchase_year_2016,purchase_year_2017,purchase_year_2018,purchase_year_2019,purchase_month_1,purchase_month_3,purchase_month_4,purchase_month_5,purchase_month_6,...,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12,purchase_timestamp,release_timestamp
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0.853261,0.564352
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.655231,0.693480
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0.853261,0.574725
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.655910,0.278577
4,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0.812500,0.458933


In [15]:
time_interval = date_test['release_date'].map(lambda x: 2020-x.year)
print(time_interval.describe())
time_interval.to_frame()

count    447.000000
mean       5.055928
std        2.631988
min        1.000000
25%        3.000000
50%        4.000000
75%        7.000000
max       14.000000
Name: release_date, dtype: float64


,release_date
0,7
1,5
2,6
3,10
4,8
...,...
442,7
443,11
444,7
445,3


In [16]:
# Dummy variables for is_free
Is_free = pd.DataFrame()
Is_free["is_free"] = pd.Series( np.where( full.is_free == True , 1 , 0 ) , name = 'is_free' )

In [17]:
Genres = pd.DataFrame()
Genres['genres'] = full['genres']
#Genres['genres'] = train['genres']
Genres = pd.concat([Genres.drop('genres', 1), Genres['genres'].str.get_dummies(sep=",")], 1)
#Genres.drop(columns='Nudity',axis=1, inplace=True)

In [18]:
# Genres_correlation = pd.concat([Genres,full['playtime_forever']],axis=1)
# Genres_correlation = Genres_correlation[:train.shape[0]]
# plt.figure(figsize=(12,10))
# cor = Genres_correlation.corr()
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# plt.show()
# cor_target = abs(cor["playtime_forever"])
# #Selecting highly correlated features
# relevant_features = cor_target[cor_target>0.5]
# relevant_features

In [19]:
Categories = pd.DataFrame()
Categories['categories'] = full['categories']
#Categories['categories'] = train['categories']
Categories = pd.concat([Categories.drop('categories', 1), Categories['categories'].str.get_dummies(sep=",")], 1)

In [20]:
Tags = pd.DataFrame()
Tags['tags'] = full['tags']
#Tags['tags'] = train['tags']
Tags = pd.concat([Tags.drop('tags', 1), Tags['tags'].str.get_dummies(sep=",")], 1)

In [21]:
tags_corr = pd.concat([Genres,Categories,Tags],axis=1)
tags_corr.shape

(447, 366)

In [22]:
corr_matrix = tags_corr.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
#to_drop

In [23]:
#dont drop now, will drop both columns with relationship
# for x in to_drop:
#     if x in tags_corr.columns:
#         tags_corr.drop(columns=str(x),axis=1, inplace=True)

In [24]:
# Genres = pd.DataFrame(np.zeros((full.shape[0], 1)))
# for x in range(full.shape[0]):
#     genres = full['genres'][x].split(',')
#     for y in genres:
#         y = 'genres_'+y
#         if y not in Genres.columns:
#             Genres[y] = 0
#         Genres[y][x] = 1
# #drop the created 0 column
# Genres = Genres.drop([0],axis=1)
# Genres.head()

In [25]:
#timestamp.min()

In [26]:
Total_positive_reviews = pd.DataFrame()
Total_positive_reviews['total_positive_reviews'] = np.log((full["total_positive_reviews"]+1)/time_interval)
Total_positive_reviews.head()

,total_positive_reviews
0,3.975668
1,1.568616
2,6.220921
3,8.749558
4,7.007827


In [27]:
Total_negative_reviews = pd.DataFrame()
Total_negative_reviews['total_negative_reviews'] = np.log((full["total_negative_reviews"]+1)/time_interval)
Total_negative_reviews.head()

,total_negative_reviews
0,2.628801
1,-1.609438
2,4.706523
3,5.163070
4,4.182050


In [28]:
#price
Price = pd.DataFrame()
Price["price"] = np.log(full["price"]+1)
Price.head()

,price
0,8.216358
1,0.000000
2,8.517393
3,9.200391
4,8.476580


In [29]:
full_XY = pd.concat( [ Price, tags_corr,date, Total_positive_reviews , Total_negative_reviews, full['playtime_forever'] ] , axis=1 )
full_XY = full_XY.loc[:,~full_XY.columns.duplicated()]
full_X = full_XY.loc[:,'price':'total_negative_reviews']
full_train = full_XY[:357]
full_test = full_XY[357:]

train_X = full_train.loc[:,'price':'total_negative_reviews']
train_y = full_train['playtime_forever']
test_X = full_test.loc[:,'price':'total_negative_reviews']
test_y = full_test['playtime_forever']

full_train.head()

,price,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Free to Play,Gore,...,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12,purchase_timestamp,release_timestamp,total_positive_reviews,total_negative_reviews,playtime_forever
0,8.216358,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0.853261,0.564352,3.975668,2.628801,0.000000
1,0.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.655231,0.693480,1.568616,-1.609438,0.016667
2,8.517393,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0.853261,0.574725,6.220921,4.706523,0.000000
3,9.200391,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.655910,0.278577,8.749558,5.163070,1.533333
4,8.476580,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.812500,0.458933,7.007827,4.182050,22.333333


In [223]:
# # X, y = make_regression(n_features=4, n_informative=2,
# #                        random_state=0, shuffle=False)
# regr = RandomForestRegressor(max_depth=6, random_state=0,
#                              n_estimators=39, bootstrap =True， oob_score = True, )
# regr.fit(train_X, train_y)  






# importance = regr.feature_importances_

# print(regr.predict(test_X))

In [30]:
# len(importance[importance>0.01])
# #train_X_filter = train_X[train_X.columns[np.where(importance[importance>0.01])]]
# test_X_filter = test_X[train_X.columns[np.where(importance[importance>0.01])]]

In [31]:
data_dmatrix = xgb.DMatrix(data=train_X,label=train_y)

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=123)

In [53]:
#16.4 This is the submission result
from sklearn.model_selection import GridSearchCV
cv_params = {'learning_rate': [0.095,0.098,0.1, 0.102,0.105]}
other_params = {'learning_rate': 0.1, 'n_estimators': 39, 'max_depth': 8, 'min_child_weight': 2, 'seed': 0,
                'subsample': 0.72, 'colsample_bytree': 0.78, 'gamma': 0.11, 'reg_alpha': 0.03, 'reg_lambda': 0.97}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train_X, train_y)
# evalute_result = optimized_GBM.grid_scores_
# print('每轮迭代运行结果:{0}'.format(evalute_result))
# print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
# print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
print("Lowest RMSE found: ", np.sqrt(np.abs(optimized_GBM.best_score_)))
print("Best parameters found: ",optimized_GBM.best_params_)

a = optimized_GBM.predict(test_X)
a[a<0] = 0
print(len(a))
a

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    2.3s finished


[20:35:55] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Lowest RMSE found:  9.742091120032217
Best parameters found:  {'learning_rate': 0.1}
90


array([1.9114041e-01, 5.1005874e+00, 2.9525858e-01, 0.0000000e+00,
       5.5335937e+00, 3.0026004e-01, 1.1037109e+00, 3.0708432e-01,
       1.5925955e+00, 9.3925595e-01, 4.0547237e-01, 0.0000000e+00,
       1.4821249e+01, 5.9324324e-01, 1.1855027e+01, 1.3797539e-01,
       7.9196483e-02, 3.4473461e-01, 2.9769585e-01, 7.3555410e-01,
       7.0052373e-01, 1.4179189e+00, 8.2380950e-02, 1.8825224e-01,
       0.0000000e+00, 3.3999416e-01, 2.7229318e-01, 4.4814977e-01,
       1.3320761e+00, 2.7623296e+00, 8.2230687e-02, 1.9979580e+01,
       1.1737049e-02, 2.3031825e-01, 2.5334847e+00, 2.1999091e-02,
       3.0517325e+00, 1.7059301e+00, 1.2254424e+00, 0.0000000e+00,
       8.6613619e-01, 4.8232132e-01, 7.4355063e+00, 2.5104228e-01,
       5.2258396e-01, 1.1410849e+01, 0.0000000e+00, 4.8687470e-01,
       1.7014444e-01, 9.6109667e+00, 1.7184952e-01, 2.6830858e-01,
       2.3888052e-02, 7.9040372e-01, 2.8215439e+00, 3.2662004e-01,
       2.0398779e+00, 2.5540531e-02, 2.6591623e-01, 9.8052394e

In [54]:
submission = pd.DataFrame()
submission['playtime_forever'] = a

submission.index.name = 'id'

submission.to_csv('Xgboost.csv')

# This is the end of my submission code

In [67]:
# xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.20,
#                 max_depth = 5, alpha = 10, n_estimators = 1000)
# xg_reg.fit(X_train,y_train)

# preds = xg_reg.predict(X_test)
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE: %f" % (rmse))

[00:09:37] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 10.436773


In [68]:
# params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
#                 'max_depth': 5, 'alpha': 10}

# cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
#                     num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

[00:39:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:39:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:39:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [76]:
# cv_results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,10.907742,1.628234,10.719149,3.929887
1,10.425014,1.579067,10.704511,3.934850
2,9.905357,1.557947,10.647422,3.935909
3,9.416627,1.423491,10.433511,4.101458
4,8.974067,1.352687,10.357797,4.050873


In [70]:
# print((cv_results["test-rmse-mean"]).tail(1))

8    10.278177
Name: test-rmse-mean, dtype: float64


In [85]:
# # Create your housing DMatrix: housing_dmatrix
# housing_dmatrix = xgb.DMatrix(data=train_X, label=train_y)

# # Create the parameter grid: gbm_param_grid
# gbm_param_grid = {
#     'colsample_bytree': [0.3, 0.7],
#     'n_estimators': [50],
#     'max_depth': [2, 5]
# }

# # Instantiate the regressor: gbm
# gbm = xgb.XGBRegressor()

In [86]:
# gbm.fit(X_train,y_train)

[10:58:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [87]:
# preds = gbm.predict(X_test)
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE: %f" % (rmse))

RMSE: 11.673498


In [33]:
# from sklearn.model_selection import RandomizedSearchCV

# gbm_param_grid = {
#     'learning_rate': [0.1,0.2,0.3,0.4,0.5],
#     'n_estimators': range(10,200),
#     'max_depth': range(2, 10)
# }

# # Instantiate the regressor: gbm
# gbm = xgb.XGBRegressor(n_estimators=100)

# # Perform random search: grid_mse
# randomized_mse = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_param_grid,
#                                     n_iter=5, scoring='neg_mean_squared_error', cv=4, verbose=1)
# randomized_mse.fit(X_train,y_train)

# # Print the best parameters and lowest RMSE
# print("Best parameters found: ",randomized_mse.best_params_)
# print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

In [ ]:
# from sklearn.model_selection import GridSearchCV
# cv_params = { 'learning_rate': [0.088,0.089,0.09,0.091,0.092]}
# other_params = {'learning_rate': 0.09, 'n_estimators': 10, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
#                     'subsample': 0.92, 'colsample_bytree': 0.77, 'gamma': 0.69, 'reg_alpha': 0.93, 'reg_lambda': 0.06}

# model = xgb.XGBRegressor(**other_params)
# optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
# optimized_GBM.fit(train_X_filter, train_y)
# # evalute_result = optimized_GBM.grid_scores_
# # print('每轮迭代运行结果:{0}'.format(evalute_result))
# # print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
# # print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
# print("Lowest RMSE found: ", np.sqrt(np.abs(optimized_GBM.best_score_)))
# print("Best parameters found: ",optimized_GBM.best_params_)

In [559]:
# corr_matrix = full_train.corr().abs()
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
# to_drop
# for x in to_drop:
#    if x in full_X.columns:
#        full_X.drop(columns=str(x),axis=1, inplace=True)

In [46]:
# plt.figure(figsize=(12,10))
# cor = full_train.corr()
# # sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# # plt.show()
# cor_target = abs(cor["playtime_forever"])
# #Selecting highly correlated features
# relevant_features = cor_target[cor_target>0.15]
# relevant_features

In [47]:
# cor = full_train[relevant_features.index].corr()
# corr_matrix = full_train[relevant_features.index].corr().abs()
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# to_drop = [column for column in upper.columns if any(upper[column] > 0.70)]
# to_drop

In [48]:
# relevant_features = relevant_features.drop('Co-op')
# relevant_features = relevant_features.drop('playtime_forever')
# relevant_features = relevant_features.drop('total_negative_reviews')

In [49]:
# full_X = full_X[relevant_features.index]
# train_X = full_X[:357]
# train_y = full_train['playtime_forever']
# test_X = full_X[357:]
# test_y = full_test['playtime_forever']

In [50]:
# #full_X = pd.concat( [ Is_free , Price, Total_positive_reviews , Total_negative_reviews ] , axis=1 )
# #full_X.head()
# pca = decomposition.PCA(n_components=3)
# full_X = pca.fit_transform(full_X)
# explained_variance = pca.explained_variance_ratio_
# explained_variance

In [51]:
# #trainX, ValidationX,trainY ,ValidationY = train_test_split(train_X, train_y, test_size=0.2)
# train, test, target, target_test = train_test_split(train_X, train_y,test_size = .2, random_state=0)

In [37]:
# train_X.corr()
# cor = full_train.corr()
# corr_matrix = full_train.corr().abs()
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# to_drop = [column for column in upper.columns if any(upper[column] > 0.70)]
# to_drop